In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.tools import TavilySearchResults

tavily_search_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # include_domains=[...],
    # exclude_domains=[...],
    # name="...",            # overwrite default tool name
    #description="search action을 수행합니다.",     # overwrite default tool description
    # args_schema=...,       # overwrite default args_schema: BaseModel
)
    # 기업의 주요 사업부와 그와 관련된 뉴스를 확인합니다.
    # 사업부별 매출처와 관련된 시장 전망 뉴스를 수집합니다.
    # 사업부별 매출처와 관련된 기업의 경쟁 상황 관련 뉴스를 확인합니다.
    # 사업부별 매출처와 관련된 기업의 성장 가능성 관련 뉴스를 확인합니다.
    # 사업부별 매출처와 관련된 기업의 위험 요소 관련 뉴스를 확인합니다.

In [3]:
from langchain_core.tools import tool

# @tool
# def major_revenue_source_tool(company: str) -> str:
#     """기업의 주요 매출처를 재무제표 검색을 통해 확인합니다."""
#     if company == "네이버":
#         return {"주요 매출처":["서치플랫폼", "커머스","클라우드"]}
#     else:
#         return None


In [4]:
import yfinance as yf
from yahooquery import search
from deep_translator import GoogleTranslator

def get_ticker(company_name):
    try:
        # 사전 매핑 확인
        # if company_name in manual_mapping:
        #     return manual_mapping[company_name]
        
        # 회사명을 영어로 번역
        translated = GoogleTranslator(source='auto', target='en').translate(company_name)
        # print(f"Translated company name: {translated}")
        
        # 번역된 이름으로 검색
        results = search(translated)
        if results is not None and 'quotes' in results and len(results['quotes']) > 0:
            # 첫 번째 결과의 티커 반환
            return results['quotes'][0]['symbol']
        else:
            print(f"No ticker found for {company_name} after translation.")
            return None
    except Exception as e:
        print(f"Error translating or searching for {company_name}: {e}")
        return None

@tool
def find_price_tool(company: str) -> str:
    """기업의 어제 종가를 찾습니다."""
    ticker = get_ticker(company)
    if ticker is None:
        return None 
    else:
        ticker = yf.Ticker(ticker)
        last_price = ticker.info["regularMarketPreviousClose"]
        return {"어제 종가":last_price}
    
@tool
def find_PER_tool(company: str) -> str:
    """기업의 현재 PER를 찾습니다."""
    ticker = get_ticker(company)
    if ticker is None:
        return None
    else:
        ticker = yf.Ticker(ticker)
        PER = ticker.info["forwardPE"]
        return {"PER":PER}
    
@tool
def find_EPS_tool(company: str) -> str:
    """기업의 현재 EPS을 yfinance를 통해 확인하고 계산합니다."""
    ticker = get_ticker(company)
    if ticker is None:
        return None
    else:
        ticker = yf.Ticker(ticker)
        EPS = ticker.info["netIncomeToCommon"]/ticker.info["sharesOutstanding"]
        return {"EPS":EPS}


In [5]:
from langchain_openai import ChatOpenAI
sub_llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [6]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

In [7]:
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변, 파이썬 리스트 형식이어야 함."),
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [8]:
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [9]:
chain = prompt | sub_llm | output_parser  # 프롬프트, 모델, 출력 파서를 연결

In [10]:
def find_peer(company: str) -> list[str]:
    prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
    )
    chain = prompt | sub_llm | output_parser  # 프롬프트, 모델, 출력 파서를 연결
    peer_list = chain.invoke({"question": f"{company}와 사업구조가 비슷하고, 같은 산업 혹은 섹터에 속한 경쟁사는?"})
    return peer_list


@tool
def find_peer_PERs_tool(company: str):
    """기업과 동종 업계의 Peer Group PER 평균을 찾습니다."""
    ticker = get_ticker(company)
    peer_list = find_peer(company)
    if ticker is None:
        return None
    
    peer_pers = {}
    for peer in peer_list:
        ticker = get_ticker(peer)
        if ticker is None:
            continue
        elif ".KS" in ticker:
            ticker = yf.Ticker(ticker)
            per = ticker.info.get("forwardPE")
            peer_pers[peer] = per
        else:
            ticker = yf.Ticker(ticker)
            per = ticker.info.get("forwardPE")*0.7 # 외국 주식의 경우 PER을 30% 할인
            peer_pers[peer] = per
    
    average_peer_per = sum(peer_pers.values()) / len(peer_pers)

    return {
        "Peer PERs": peer_pers,
        "Peer list": peer_list,
        "Average Peer PER": average_peer_per
    }

In [11]:
from langgraph.prebuilt import create_react_agent
from datetime import datetime

today = datetime.now().strftime("%Y-%m-%d")
#TODO: PER로 계산한 결과가 뉴스로 찾은 근거와는 다르게 나오는 경우 PBR 계산으로 확인하도록 하거나, 처음부터 기업 상황을 체크해서 PER, PBR, EV/EBITDA 중 하나를 택하도록 하는 방법 있음.
prompt = f"""항상 {today}를 기준으로 확인 및 예상합니다.
기업의 매출을 책임지는 주요 사업부 리스트와 주요 사업부의 수익 창출 방법 설명 그리고 그 사업부의 매출 비중을 확인.
사업부별로 매출이나 이익에 주요한 영향을 미칠 수 있는 뉴스(사업부 관련 뉴스)를 확인.
한국 경제에 영향을 미치기 때문에 사업부 매출과 이익에 영향을 줄 수 있는 거시 경제 관련 뉴스(거시 경제 뉴스)를 확인.
한국 경제 혹은 기업에 영향을 미치기 때문에 사업부 매출과 이익에 영향을 줄 수 있는 정치 관련 뉴스(정치 뉴스)를 확인
사업부 관련 뉴스, 거시 경제 뉴스, 정치 뉴스를 기반으로 사업부별 매출을 예상.
예상한 사업부별 매출을 기반으로 기업의 목표 PER을 설정
(사업부별 매출이 좋을 것으로 예상되거나, 매출 성장 가능성이 높으면 목표 PER을 높게 설정,
사업부별 매출이 나쁠 것으로 예상되거나, 매출 성장 가능성이 낮으면 목표 PER을 낮게 설정,
peer PER 평균을 목표 PER과 같이 보여주기, 
peer PER 평균을 이루는 peer가 어떤 기업인지 항상 명시).
목표 PER을 현재 EPS를 곱하여 목표 주가를 설정.
목표 주가를 어제 종가와 비교.
모든 정보를 토대로 기업의 가치를 평가하고 투자의견 제시.
"""
tools = [tavily_search_tool, find_price_tool, find_PER_tool, find_peer_PERs_tool, find_EPS_tool]

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

llm.bind_tools(tools)

graph = create_react_agent(llm, tools=tools,state_modifier=prompt)

In [12]:
inputs = {"messages": [("user", "삼성전자")]}
for chunk in graph.stream(inputs, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

삼성전자
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_7KXwO8apen38zMrgyMaHHTvL)
 Call ID: call_7KXwO8apen38zMrgyMaHHTvL
  Args:
    query: 삼성전자 주요 사업부 및 매출 비중 2024
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://news.samsungsemiconductor.com/kr/삼성전자-2024년-1분기-실적-발표/", "content": "삼성전자는 연결 기준으로 매출 71.92조원, 영업이익 6.61조원의 2024년 1분기 실적을 발표했다. 전사 매출은 플래그십 스마트폰 갤럭시 S24 판매 호조 및 메모리 시황 개선에 따른 판가 상승으로 전분기 대비 6% 증가한 71.92조원을 기록했다."}, {"url": "https://news.samsung.com/kr/삼성전자-2024년-2분기-실적-발표", "content": "삼성전자는 연결 기준으로 매출 74.07조원, 영업이익 10.44조원의 2024년 2분기 실적을 발표했다. 전사 매출은 전분기 대비 3% 증가한 74.07조원을 기록했다. DS부문은 메모리 업황 회복으로 전분기 대비 23% 증가하고, SDC는 OLED 판매 호조로 증가했다. 영업이익은"}, {"url": "https://news.samsung.com/kr/삼성전자-2024년-1분기-실적-발표", "content": "삼성전자는 연결 기준으로 매출 71.

In [13]:
inputs = {"messages": [("user", "네이버")]}
for chunk in graph.stream(inputs, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

네이버
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_3jQPlikxmpFklfaio7q9k7rL)
 Call ID: call_3jQPlikxmpFklfaio7q9k7rL
  Args:
    query: 네이버 주요 사업부 및 매출 비중 2024
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.datanews.co.kr/news/article.html?no=134042", "content": "네이버(대표 최수연)는 2024년 2분기에 매출 2조6105억 원, 영업이익 4727억 원을 기록했다고 9일 밝혔다.2분기 연결 매출은 서치플랫폼, 커머스, 핀테크 등 주요 사업 부문의 고른 성장과 클라우드 매출 증대로 전년 동기 대비 8.4% 증가했다. 연결 영업이익은 전년 동.."}, {"url": "https://www.g-enews.com/article/ICT/2024/05/202405030838022700ea588b1547_1", "content": "네이버는 2024년 1분기에 매출액 2조5261억 원, 조정 EBITDA 5810억 원, 영업이익 4393억 원을 각각 기록했다고 3일 밝혔다. 주요 사업 부문의 고른 성장이"}, {"url": "https://biz.newdaily.co.kr/site/data/html/2024/05/03/2024050300100.html", "content": "최수연 네이버 대표가 1분기 주요 사업 부문의 고

In [14]:
inputs = {"messages": [("user", "SK하이닉스")]}
for chunk in graph.stream(inputs, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

SK하이닉스
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_WIikP2KCcDG2Fn8ILw9rOzmX)
 Call ID: call_WIikP2KCcDG2Fn8ILw9rOzmX
  Args:
    query: SK하이닉스 주요 사업부 및 매출 비중 2024
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.sks.co.kr/data1/research/qna_file/20240413212539503_0_ko.pdf", "content": "sk 하이닉스 실적 추정 주요 ... 자료: sk증권 추정 sk 하이닉스 2024년 분기 실적 전망 ... 매출채권 및 기타채권 5,444 6,942 11,123 12,680 12,683 매출총이익 15,628 -533 25,946 37,903 44,783 재고자산 15,665 13,481 14,271 11,648 13,782 매출총이익률(%) 35.0 -1.6 39.9 46.5 50.0"}, {"url": "https://m.blog.naver.com/meowk/223632588094", "content": "sk하이닉스 2024 3분기 실적 요약: - 매출: 17조5731억원 (전년 동기 대비 93.8% 증가) ... ai 반도체 수요 증가, hbm(고대역폭메모리) 시장 장악 2. 주요 실적 배경: - ai 학습 및 데이터센터 수요 증가로 hbm과 essd(고성능 저장 장치) 판매 급증 - d램 매출 비중: